In [1]:
%run init.ipynb

C:\Users\Administrator\Anaconda3\lib\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.11) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


matchzoo version 1.0.2
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
停用表配置成功


In [2]:
preprocessor = mz.models.CDSSM.get_default_preprocessor(
    ngram_size = 3
)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval:   0%| | 0/95 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 2.607 seconds.
Prefix dict has been built succesfully.
Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 95/95 [00:03<00:00, 27.35it/s]
Processing text_right with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 97/97 [00:00<00:00, 127.88it/s]
Building Vocabulary from a datapack.: 100%|██████████████████████████████████████| 8

In [4]:
preprocessor.context

{'embedding_input_dim': 307,
 'filter_unit': <mzcn.preprocessors.units.frequency_filter.FrequencyFilter at 0x1af4c9dfb00>,
 'ngram_process_unit': <mzcn.preprocessors.units.ngram_letter.NgramLetter at 0x1af5a4fcbe0>,
 'ngram_vocab_size': 307,
 'ngram_vocab_unit': <mzcn.preprocessors.units.vocabulary.Vocabulary at 0x1af5a6f9cc0>,
 'vocab_size': 307,
 'vocab_unit': <mzcn.preprocessors.units.vocabulary.Vocabulary at 0x1af5a6f4a58>}

In [5]:
triletter_callback = mz.dataloader.callbacks.Ngram(preprocessor, mode='sum')
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    callbacks=[triletter_callback]
)
devset = mz.dataloader.Dataset(
    data_pack=dev_pack_processed,
    callbacks=[triletter_callback]
)

In [6]:
padding_callback = mz.models.CDSSM.get_default_padding_callback(
    with_ngram=True,
    fixed_ngram_length=preprocessor.context['ngram_vocab_size']
)

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback,
)
devloader = mz.dataloader.DataLoader(
    dataset=devset,
    stage='dev',
    callback=padding_callback,
)

In [7]:
model = mz.models.CDSSM()

model.params['task'] = ranking_task
model.params['vocab_size'] = preprocessor.context['ngram_vocab_size']
model.params['filters'] = 64
model.params['kernel_size'] = 3
model.params['conv_activation_func'] = 'tanh'
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 64
model.params['mlp_num_fan_out'] = 64
model.params['mlp_activation_func'] = 'tanh'
model.params['dropout_rate'] = 0.8

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

CDSSM(
  (net_left): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(307, 64, kernel_size=(3,), stride=(1,))
    (2): Tanh()
    (3): Dropout(p=0.8, inplace=False)
    (4): AdaptiveMaxPool1d(output_size=1)
    (5): Squeeze()
    (6): Sequential(
      (0): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
    )
  )
  (net_right): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(307, 64, kernel_size=(3,), stride=(1,))
    (2): Tanh()
    (3): Dropout(p=0.8, inplace=False)
    (4): AdaptiveMaxPool1d(output_size=1)
    (5): Squeeze()
    (6): Sequential(
      (0): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tan

In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4)

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=devloader,
    validate_interval=None,
    epochs=15
)

trainer.run()

[Iter-1 Loss-1.040]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2647 - normalized_discounted_cumulative_gain@5(0.0): 0.2647 - mean_average_precision(0.0): 0.2647



[Iter-2 Loss-1.061]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2647 - normalized_discounted_cumulative_gain@5(0.0): 0.2647 - mean_average_precision(0.0): 0.2647



[Iter-3 Loss-0.983]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2647 - normalized_discounted_cumulative_gain@5(0.0): 0.2647 - mean_average_precision(0.0): 0.2647



[Iter-4 Loss-0.999]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2647 - normalized_discounted_cumulative_gain@5(0.0): 0.2647 - mean_average_precision(0.0): 0.2647



[Iter-5 Loss-0.996]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2647 - normalized_discounted_cumulative_gain@5(0.0): 0.2647 - mean_average_precision(0.0): 0.2647



...

[Iter-11 Loss-1.034]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2647 - normalized_discounted_cumulative_gain@5(0.0): 0.2647 - mean_average_precision(0.0): 0.2647



[Iter-12 Loss-0.980]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2647 - normalized_discounted_cumulative_gain@5(0.0): 0.2647 - mean_average_precision(0.0): 0.2647



[Iter-13 Loss-0.971]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2647 - normalized_discounted_cumulative_gain@5(0.0): 0.2647 - mean_average_precision(0.0): 0.2647



[Iter-14 Loss-0.891]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2647 - normalized_discounted_cumulative_gain@5(0.0): 0.2647 - mean_average_precision(0.0): 0.2647



[Iter-15 Loss-1.001]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2647 - normalized_discounted_cumulative_gain@5(0.0): 0.2647 - mean_average_precision(0.0): 0.2647

Cost time: 6.521260976791382s


In [9]:
import gc
gc.collect()

654